In [ ]:
%matplotlib inline

In [ ]:
!pip3 install opencv-python

In [ ]:
# Import OpenCV2 for image processing
import cv2

# Start capturing video 
vid_cam = cv2.VideoCapture(0)

# Detect object in video stream using Haarcascade Frontal Face
face_detector = cv2.CascadeClassifier('C:\\Users\\hp pc\\Downloads\\Raspberry-Face-Recognition-master (1)\\Raspberry-Face-Recognition-master\\haarcascade_frontalface_default.xml')

# For each person, one face id
face_id =2 

# Initialize sample face image
count = 0

# Start looping
while(True):

    # Capture video frame
    _, image_frame = vid_cam.read()

    # Convert frame to grayscale
    gray = cv2.cvtColor(image_frame, cv2.COLOR_BGR2GRAY)

    # Detect frames of different sizes, list of faces rectangles
    faces = face_detector.detectMultiScale(gray, 1.3, 5)

    # Loops for each faces
    for (x,y,w,h) in faces:

        # Crop the image frame into rectangle
        cv2.rectangle(image_frame, (x,y), (x+w,y+h), (255,0,0), 2)
        
        # Increment sample face image
        count += 1

        # Save the captured image into the datasets folder
        cv2.imwrite("C:\\Users\\hp pc\\Downloads\\dataset\\" + str(face_id) + '.' + str(count) + ".jpg", gray[y:y+h,x:x+w])

        # Display the video frame, with bounded rectangle on the person's face
        cv2.imshow('frame', image_frame)

    # To stop taking video, press 'q' for at least 100ms
    if cv2.waitKey(100) & 0xFF == ord('q'):
        break

    # If image taken reach 100, stop taking video
    elif count>=200:
        break

# Stop video
vid_cam.release()

# Close all started windows
cv2.destroyAllWindows()


In [ ]:
import cv2
from time import time
import logging
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from sklearn.svm import SVC


print(__doc__)

# Display progress logs on stdout
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(message)s')


# #############################################################################
                          1           2           3
import glob
import re #regex

data = []                   
labs = []

for name in glob.glob('C:\\Users\\hp pc\\Downloads\\dataset*\\*.jpg'):
    img = cv2.imread(name)
    img = cv2.resize(img, (128,128))
    data.append(img.flatten())
    lab = re.search(r'\\([a-zA-Z0-9]+)\\([a-zA-Z0-9.]+).jpg', name).group(2)
    labs.append(lab)
    #print(name)
    #print(lab)
print(img.shape)
#print(labs)
print(data)
print('Number of files read:', len(data))

In [ ]:
import numpy as np
X = np.array(data)
y = np.array(labs).reshape(-1,1)
print(y)


In [ ]:
# #############################################################################
# Split into a training set and a test set using a stratified k fold

# split into a training and testing set
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42)


# #############################################################################
# Compute a PCA (eigenfaces) on the face dataset (treated as unlabeled
# dataset): unsupervised feature extraction / dimensionality reduction
n_components = 300

print("Extracting the top %d eigenfaces from %d faces"
      % (n_components, X_train.shape[0]))
t0 = time()
pca = PCA(n_components=n_components, svd_solver='randomized',
          whiten=True).fit(X_train)
print("done in %0.3fs" % (time() - t0))

print("Projecting the input data on the eigenfaces orthonormal basis")
t0 = time()
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)
print("done in %0.3fs" % (time() - t0))

# #############################################################################
# Train a SVM classification model
from sklearn.svm import SVC
svclassifier = SVC(kernel='linear', degree=8)
svclassifier.fit(X_train, y_train)

In [ ]:

# Import OpenCV2 for image processing
# Import os for file path
import cv2, os

# Import numpy for matrix calculation
import numpy as np

# Import Python Image Library (PIL)
from PIL import Image

# Create Local Binary Patterns Histograms for face recognization
recognizer = cv2.face.LBPHFaceRecognizer_create()

# Using prebuilt frontal face training model, for face detection
detector = cv2.CascadeClassifier("C:\\Users\\hp pc\\Downloads\\Raspberry-Face-Recognition-master (1)\\Raspberry-Face-Recognition-master\\haarcascade_frontalface_default.xml");

# Create method to get the images and label data
def getImagesAndLabels(path):

    # Get all file path
    imagePaths = [os.path.join(path,f) for f in os.listdir(path)] 
    
    # Initialize empty face sample
    faceSamples=[]
    
    # Initialize empty id
    ids = []

    # Loop all the file path
    for imagePath in imagePaths:

        # Get the image and convert it to grayscale
        PIL_img = Image.open(imagePath).convert('L')

        # PIL image to numpy array
        img_numpy = np.array(PIL_img,'uint8')

        # Get the image id
        id = int(os.path.split(imagePath)[-1].split(".")[0])
        print(id)

        # Get the face from the training images
        faces = detector.detectMultiScale(img_numpy)
        print(faces)

        # Loop for each face, append to their respective ID
        for (x,y,w,h) in faces:

            # Add the image to face samples
            faceSamples.append(img_numpy[y:y+h,x:x+w])

            # Add the ID to IDs
            ids.append(id)

    # Pass the face array and IDs array
    return faceSamples,ids

# Get the faces and IDs
faces,ids = getImagesAndLabels('C:\\Users\\hp pc\\Downloads\\dataset\\')

# Train the model using the faces and IDs
recognizer.train(faces, np.array(ids))

# Save the model into trainer.yml
recognizer.save('D:\\trainer.yml')


In [ ]:
# Import OpenCV2 for image processing
import cv2

# Import numpy for matrices calculations
import numpy as np

# Create Local Binary Patterns Histograms for face recognization
recognizer = cv2.face.LBPHFaceRecognizer_create()

# Load the trained mode
recognizer.read('D:\\trainer.yml')

# Load prebuilt model for Frontal Face
cascadePath = "C:\\Users\\hp pc\\Downloads\\Raspberry-Face-Recognition-master (1)\\Raspberry-Face-Recognition-master\\haarcascade_frontalface_default.xml"

# Create classifier from prebuilt model
faceCascade = cv2.CascadeClassifier(cascadePath);

# Set the font style
font = cv2.FONT_HERSHEY_SIMPLEX

# Initialize and start the video frame capture
cam = cv2.VideoCapture(0)

# Loop
while True:
    # Read the video frame
    ret, im =cam.read()

    # Convert the captured frame into grayscale
    gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)

    # Get all face from the video frame
    faces = faceCascade.detectMultiScale(gray, 1.2,5)

    # For each face in faces
    for(x,y,w,h) in faces:
        Id, conf = recognizer.predict(gray[y:y+h,x:x+w])
        cv2.rectangle(im,(x-50,y-50),(x+w+50,y+h+50),(225,0,0),3)
        if(Id==2):
             Id='priya'
        elif(Id == 3):
            Id = "HARISH"
        elif(Id == 4):
            Id = "SOJA "    
                
        else:
             Id='UNKNOWN'
        cv2.putText(im,str(Id)+"--"+str(conf), (x,y+h),font, 1.1, (0,255,0),3) #Draw the text
       # cv2.putText(im, str(Id), (x,y-40), font, 2, (255,255,255), 3)
        cv2.imshow('im',im)
        print(Id,conf)
    # If 'q' is pressed, close program
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Stop the camera
cam.release()

# Close all windows
cv2.destroyAllWindows()


In [ ]:
for khjjhg in range(10):
    print('soja')